In [ ]:
!nvidia-smi

In [ ]:
%run rl_rebuilt.ipynb
%run env_rebuilt_simple.ipynb
# %run env_final_reward_scale_2data.ipynb
%run function.ipynb
# %run backtest_paramters.ipynb
%run model_rebuilt.ipynb
import math
import graphviz
import tensorflow as tf 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, LSTM, Input, Concatenate, Subtract, Reshape, Lambda, merge  ,GRU, ThresholdedReLU
from keras.optimizers import Adam
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy, BoltzmannGumbelQPolicy, GreedyQPolicy
from keras.callbacks import TensorBoard
import talib 
from IPython.display import SVG 
from keras.utils.vis_utils import model_to_dot
import csv
import random

In [ ]:
"""
DATA_NM : 資料名稱
START_PRE_DATE : 開始預測時間
HISTORY_LENGTH : 歷史資料需求長度
TIME_DATA_LENGTH : 有時間效應之DATA長度
NOT_TIME_DATA_LENGTH : 沒有時間效應之DATA長度
TRADING_FEE : 交易手續費 (%)
TIME_FEE : 時間消耗成本 (EX:定存利率)
BUY_MIN_UNIT : 最少買進單位數
EPISODE_LENGTH : 最多step數量
NB_ACTIONS : 動作數量
REWARD_COL : 計算獎勵欄位名稱
MEMORY_LENGTH : 記憶庫裡資料的個數
NB_STEPS_WARMUP : 一開始多少step後開始train network 
TRAIN_INTERVAL : 隔多少step後隨機抽取 batch 去 train
MEMORY_INTERVL : 隔多少step後append memory
TARGET_MODEL_UPDATE : 隔多少step後更新目標 network
NN_CONSTRUCTION : network 架構
POLICY : 決策方式
value_max : random最大機率， value_min : random最小機率， value_test : 在test時候的random機率， nb_steps : 幾步之後從最大到最小
BATCH_SIZE : 每次training抓取資料個數
GAMMA : Q值得衰退因子

file_nm : 存檔檔案名稱
"""
#DATA

# DATA_NM = 'data/index_tsm_value.csv'
DATA_NM = 'data/some_market_index_ts_value.csv'
MONTH_DATA = 'data/index_tsm_value.csv'
INDEX_NM = 'MSCI ACWI NET TOTAL RETURN USD'
START_PRE_DATE = '2014-06-01'
step = 2000000
#ENV
HISTORY_LENGTH = 48
TRADING_FEE = 0.006
TIME_FEE = 0.001
EPISODE_LENGTH = 2000
NB_ACTIONS = 3
MIN_BUY_POSITION = 1
MAX_POSITION = 1
MA_WINDOWS=0
if MA_WINDOWS > 0 :
    REWARD_COL = INDEX_NM + '_MA'
else :
    REWARD_COL = INDEX_NM
REWARD_SCALE = 1000
PRICE_COL = INDEX_NM
NN_INPUT_COL = INDEX_NM+'_Return'
REWARD_GAMMA = 0.9
DATA_NOR = False
#MODEL
RANDOM_TRAIN = True
MEMORY_LENGTH = 20000
MEMORY_ALPHA = 0.6
MEMORY_BETA = 0.4
MEMORY_ANNEALED = 25000
NB_STEPS_WARMUP = 200
TRAIN_INTERVAL = 1
MEMORY_INTERVL = 1
TARGET_MODEL_UPDATE = 100
POLICY = LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',value_max=1.,value_min=.05,value_test=.00,nb_steps=500000)
BATCH_SIZE = 32
GAMMA = 0.9
REWARD_STEP = range(1,7) #如果有加入趨式為最大的趨勢月份
#flie name
import datetime
run_code_date = datetime.datetime.now().strftime("%Y-%m-%d")
file_nm = 'h5/'+ INDEX_NM +'/Rainbow_rebuilt_lump'+'_' + run_code_date 


In [ ]:
# markets
markets = pd.read_csv(DATA_NM,skiprows=1,names=['index_nm','tx_month','close'])
markets = markets.loc[markets.index_nm==INDEX_NM,['tx_month','close']].rename(columns={'close':INDEX_NM})
markets.tx_month = pd.to_datetime(markets.tx_month)
if  MA_WINDOWS > 0 :
    markets[INDEX_NM+'_MA'] = markets.loc[:,INDEX_NM]\
    .rolling(window=MA_WINDOWS).mean()\
    .shift(-1*int(math.floor(float(MA_WINDOWS)/2))).tolist()
markets[INDEX_NM+'_Return'] = markets.loc[:,INDEX_NM] / markets.loc[:,INDEX_NM].shift(22)  - 1
markets=markets.fillna(method='ffill').fillna(method='bfill')
markets = markets.reset_index(drop=True)

In [ ]:
# markets month
markets_month = pd.read_csv(MONTH_DATA,skiprows=1,names=['index_nm','tx_month','close']).sort_values('tx_month')
markets_month = markets_month.loc[markets_month.index_nm==INDEX_NM,['tx_month','close']].rename(columns={'close':INDEX_NM})
markets_month.tx_month = pd.to_datetime(markets_month.tx_month)
if  MA_WINDOWS > 0 :
    markets_month[INDEX_NM+'_MA'] = markets_month.loc[:,INDEX_NM]\
    .rolling(window=MA_WINDOWS).mean()\
    .shift(-1*int(math.floor(float(MA_WINDOWS)/2))).tolist()
markets_month[INDEX_NM+'_Return'] = markets_month.loc[:,INDEX_NM]/ markets_month.loc[:,INDEX_NM].shift(1)  - 1
markets_month = markets_month.fillna(method='ffill').fillna(method='bfill')
markets_month = markets_month.reset_index(drop=True)


In [ ]:
train_data_list=[]
test_data_list=[]
for i in range(22):
    monthly = data_row_gap(data=markets[i:],gap=22,miss_fill='ffill').reset_index()
    train_data,test_data = split_train_test_data(data=monthly, start_pre_date=START_PRE_DATE, 
                                                 history_length=HISTORY_LENGTH, trend_months=REWARD_STEP)
    train_data_list.append(train_data)
    test_data_list.append(test_data) 

In [ ]:
train_data_month,test_data_month = split_train_test_data(data=markets_month, start_pre_date=START_PRE_DATE, 
                                             history_length=HISTORY_LENGTH, trend_months=REWARD_STEP)

In [ ]:
no_time_var_list = []
var_list = [NN_INPUT_COL]
# var_list=name_list
TIME_VAR = len(var_list)
TIME_DATA_LENGTH = HISTORY_LENGTH*TIME_VAR
NOT_TIME_DATA_LENGTH = 2 +len(no_time_var_list)
data_list_col = np.unique(var_list+no_time_var_list+[REWARD_COL]+[PRICE_COL]).tolist()
# train_data_list=[train_data_list[0]]
params={'reward_step':[1], 'data_nor':False,
        'min_buy_position':1,'cash_raise':1000,'cash_start':3000, 'max_position':float('inf'), 'episode_length':1000,
        'trading_fee':0, 'time_fee':0, 'history_length':12,'StandardScaler_list':None,'train_lab':False,
        'data_list':[i.loc[:,data_list_col].fillna(0) for i in train_data_list],
        'trading_fee':TRADING_FEE,
        'time_fee':TIME_FEE,
        'history_length':HISTORY_LENGTH,
        'episode_length':EPISODE_LENGTH, 
        'min_buy_position':MIN_BUY_POSITION,
        'max_position':MAX_POSITION,
        'reward_col':REWARD_COL,
        'data_nor':DATA_NOR,
        'reward_step':REWARD_STEP,
        'reward_gamma':REWARD_GAMMA,
        'train_lab':RANDOM_TRAIN,
        'price_col':PRICE_COL,   
        'time_var_nm' : var_list,
        'nn_input_col': [NN_INPUT_COL],
        'no_time_var_nm' : no_time_var_list,
        'reward_scale' : REWARD_SCALE
       }
train_env = SpreadTrading_simple(params)

check_data = []
for investment_month in [36]:
    data_start_index = 0
    if REWARD_STEP:
        more_data_len = max(REWARD_STEP) + HISTORY_LENGTH + investment_month-1
    else :
        more_data_len = HISTORY_LENGTH + investment_month-1
    while data_start_index + more_data_len < len(train_data_month):
        check_data.append( train_data_month.loc[data_start_index:(data_start_index + more_data_len),data_list_col].reset_index(drop=True))
        data_start_index += 1 
SS=StandardScaler()
SS.fit(train_data_month.loc[:,[NN_INPUT_COL]])
check_params = params.copy()
check_params['data_list'] = check_data
check_params['StandardScaler_list'] = [SS]*len(check_data)
check_params['train_lab'] = False
train_check_env = SpreadTrading_simple(check_params)


In [ ]:
seed = 1337
for MEMORY_BETA in [0.2,0.4,0.6]:
    np.random.seed(seed)
    random.seed(seed)
    drl=DRL(history_length=HISTORY_LENGTH, file_nm= (file_nm+'_MEMORY_BETA_{}').format(MEMORY_BETA), \
            time_var=TIME_VAR, time_data_length=TIME_DATA_LENGTH, \
            not_time_data_length=NOT_TIME_DATA_LENGTH, nb_actions=NB_ACTIONS\
           )
    print step
    print INDEX_NM
    dqn=drl.RL_method(env=train_env,check_env=train_check_env, step=step
                      , memory_length=MEMORY_LENGTH,train_interval=TRAIN_INTERVAL,\
                      policy=POLICY, nb_steps_warmup=NB_STEPS_WARMUP,\
                      alpha=MEMORY_ALPHA,beta=MEMORY_BETA,beta_annealed=MEMORY_ANNEALED\
                      , batch_size=BATCH_SIZE, gamma=GAMMA, target_model_update=TARGET_MODEL_UPDATE)

In [ ]:
file_nm

In [ ]:
a=[1626.47998046875,1753.9100341796875, 1379.8199462890625, 1497.6199951171875]
b=[a[i+1]/a[i]-1 for i in range(len(a)-1)]
b

In [ ]:
A=[1677.4300537109375, 1973.0, 1929.6400146484373]
B=1880.1700439453125
C=[i/B-1 for i in A]
D=0.9
D*C[0]+D**2*(C[1]-C[0])+D**3*(C[2]-C[1])-0.006

In [ ]:
1677.4300537109375